## Determinação de pontos em 1D/2D dadas distâncias entre pares

Nos é apresentado o seguinte problema:

*Dada uma matriz $d_{ij} \in \mathbb{R}^{n\times n}$ indicando a distância entre cada par de pontos $i$ e $j$, o problema consiste em determinar pontos $p_1, p_2, \dots, p_n \in \mathbb{R}^2$ tentando atender as restrições $\|p_i-p_j\| = d_{ij}, \forall i, j$.*

Resolver isto analiticamente consiste na resolução de um sistema não linear no qual aparecem termos de grau 2. Este sistema pode ser resolvido usando substituição ou algum método numérico. Vale adicionar uma sessão aqui no final do trabalho sobre esta abordagem, embora o problema a seguir não seja resolvível por este método.

O problema completo então é o seguinte:

*Dada uma matriz $d_{ij} \in \mathbb{R}^{n\times n}$ indicando a distância entre cada par de pontos $i$ e $j$ em uma superfície **não necessariamente plana**, o problema consiste em determinar pontos $p_1, p_2, \dots, p_n \in \mathbb{R}^2$ tais que os valores $\|p_i-p_j\|$ e $d_{ij}$ não sejam muito diferentes, para alguma noção de diferença.*

Primeiro precisamos entender que temos $n$ pontos para escolher onde colocar no plano e que a escolha destes pontos apresenta certo grau de liberdade. De fato, dada uma configuração fixada para os pontos, qualquer translação ou aplicação linear que preserve produto interno (consequentemente preservando distâncias entre vetores, por exemplo matrizes ortogonais) resulta em outra configuração para os pontos com as mesmas distâncias entre os pontos. Mesmo não entendendo formalmente ainda o que estamos querendo descobrir, é evidente que existe certo grau de liberdade para a escolha dos pontos.

Por isso, a partir de agora, consideraremos $p_1 = \bar0$, ou seja, **$p_1$ está fixado na origem.**



### Definição da função de erro

Precisamos agora entender melhor o que estamos querendo encontrar.

Defina $p=(p_1, p_2, \dots, p_n)$. Podemos começar pensando em definir o erro de uma configuração $p$ como o seguinte valor:

$$
\displaystyle \sum_i \sum_{j>i} \left|\|p_i-p_j\|-d_{ij}\right|
$$

Esta quantidade é a soma para todo par de pontos da distância atribuída a eles pela escolha de suas coordenadas e o valor nos dado na entrada pela matriz $d_{ij}$.

Esta quantidade é significativa, mas é péssima para realizar cálculos, pois apresenta normas de vetores sem estarem ao quadrado. Por isso, a função de erro adotada daqui para frente será a seguinte:

$$
e(p) = \displaystyle \sum_i \sum_{j>i} \left|\|p_i-p_j\|^2-d_{ij}^2\right|^2
$$

Sendo assim, podemos escrever o seguinte problema de otimização, sabendo que $p_1 = \bar0$:

$$
\displaystyle \min_p e(p) = \min_p \sum_i \sum_{j>i} \left|\|p_i-p_j\|^2-d_{ij}^2\right|^2
$$

# Resolução do problema de otimização

Podemos resolver este problema de algumas maneiras. A primeira delas abordadas será utilizando o método do gradient descent (em português, gradiente descendente).

Para isto, precisamos calcular o gradiente de $e(p)$. Para facilitar os cálculos no momento, **o problema será simplificado para 1 dimensão.** Após uma primeira implementação, retornaremos para a abordagem 2D. Como estamos agora em 1D, $p_i \in \mathbb{R}$.

$\displaystyle \nabla e(p) = \left(\frac{\partial e(p)}{\partial p_1}, \frac{\partial e(p)}{\partial p_2}, \dots, \frac{\partial e(p)}{\partial p_n}\right)$

Vejamos então como calcular $\displaystyle \frac{\partial e(p)}{\partial p_k}$:

$$
\displaystyle \frac{\partial e(p)}{\partial p_k} = \frac{\partial}{\partial p_k} \left[\sum_{j>k} \left|(p_k-p_j)^2-d_{kj}^2\right|^2 + \sum_{k>i} \left|(p_i-p_k)^2-d_{ik}^2\right|^2\right]
$$

$$
\displaystyle \frac{\partial e(p)}{\partial p_k} = \frac{\partial}{\partial p_k} \sum_{i} \left|(p_i-p_k)^2-d_{ik}^2\right|^2 = 4 \sum_i (p_i-p_k)\left|(p_i-p_k)^2-d_{ik}^2\right|
$$


#### Gradient Descent

Este é um método iterativo que se aproxima cada vez mais de um mínimo local, fazendo uma dada configuração "andar" ou "dar passos" num sentido de decrescimento do valor da função. O passo iterativo é o seguinte:

$$
\begin{cases}
p^{(0)} = ? \\
p^{(k+1)} = p^{(k)} - \alpha\nabla e(p^{(k)})
\end{cases}
$$

Há duas perguntas imediatas a serem feitas:

1) Que valor escolher para $p^{(0)}$?

2) Que valor escolher para $\alpha$, que indica o tamanho do passo?

O valor de $p^{(0)}$ a princípio será escolhido aleatoriamente para cada uma das $n-1$ variáveis (visto que $p_1 = 0$).

Já o passo $\alpha$ será escolhido na mão, a partir de várias tentativas para a verificação da melhor convergência.

Segue então o código implementando o método do gradient descent para $p_i \in \mathbb{R}$.

In [62]:
import numpy as np

n = 5 # número de pontos
alpha = 0.1 # tamanho do passo
tol = 1e-8


In [63]:
D = [[.0, .7, .1, .4, .9],
     [.7, .0, .4, .3, .6],
     [.1, .4, .0, .8, .3],
     [.4, .3, .8, .0, .5],
     [.9, .6, .3, .5, .2]]

In [64]:
def gradient(p):
    grad = np.zeros(n)
    for i in range(1, n):
        for j in range(n):
            diff = p[i] - p[j]
            grad[i] += diff * ((diff**2) - D[i][j])
        grad[i] *= 4
    return grad
            

In [65]:
def erro(p):
    ret = 0
    for i in range(n):
        for j in range(i+1, n):
            ret += ((p[i]-p[j])**2 - D[i][j]**2)**2
    return ret

In [90]:
last = np.random.rand(n)
last[0] = 0
print(last)
diff_norm = float('inf')

while diff_norm > tol:
    p = last - alpha * gradient(last)
    diff_norm = np.linalg.norm(p-last)
    last = p
    
print(last, erro(last))

[ 0.          0.41801925  0.11595963  0.75242584  0.0710389 ]
[ 0.          0.7672003  -0.0169366   0.77237768  0.07609404] 1.13804024823
